In [2]:
import timm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim import Adam
from tqdm import tqdm

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图像大小
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

# 加载训练集和验证集
train_dataset = datasets.ImageFolder('C:/Users/m1522/Desktop/Working/CS4487gp/AIGC-Detection-Dataset/train', transform=transform)
val_dataset = datasets.ImageFolder('C:/Users/m1522/Desktop/Working/CS4487gp/AIGC-Detection-Dataset/val', transform=transform)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# 类别数量
num_classes = len(train_dataset.classes)
print(f"Number of classes: {num_classes}")

D:\Coding\pythons\pytorchgpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of classes: 2


In [3]:
model_name = "vit_base_patch16_224"  # 使用 ViT-B/16 模型
model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)  # 修改分类头以适应新任务

# 将模型移动到 GPU（如可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Step 3: 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()  # 交叉熵损失
optimizer = Adam(model.parameters(), lr=1e-4)  # Adam 优化器

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()  # 设置为训练模式
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(dataloader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)  # 数据移动到 GPU（如可用）

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        # 计算准确率
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    accuracy = correct / total
    return running_loss / len(dataloader), accuracy


# Step 5: 模型验证
def validate(model, dataloader, criterion, device):
    model.eval()  # 设置为评估模式
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Validating"):
            inputs, labels = inputs.to(device), labels.to(device)

            # 前向传播
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # 计算准确率
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    return running_loss / len(dataloader), accuracy

In [4]:
import matplotlib.pyplot as plt

num_epochs = 10
best_accuracy = 0.0
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss,train_accuracy = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    
    # Append accuracies for plotting
    train_accuracies.append(train_accuracy)  # Assuming train_loss is a proxy for accuracy
    val_accuracies.append(val_accuracy)
    
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), "best_vit_model.pth")
        print("Saved best model!")

print("Training complete. Best validation accuracy: {:.4f}".format(best_accuracy))

# Plotting the accuracies
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()


Epoch 1/10


Training:   0%|                                                                               | 0/1407 [00:59<?, ?it/s]


AssertionError: Input height (128) doesn't match model (224).

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(), 
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5], 
        std=[0.5, 0.5, 0.5] 
    )
])

# Step 2: 加载保存的模型
model_name = "vit_base_patch16_224"
num_classes = 10 

model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)

model.load_state_dict(torch.load("best_vit_model.pth"))
model.eval()

image_path = "example.jpg" 
image = Image.open(image_path).convert("RGB")  
input_tensor = transform(image).unsqueeze(0)  

with torch.no_grad(): 
    output = model(input_tensor)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)


class_labels = train_dataset.classes  # 训练时的类别名称列表

# 获取 Top-1 类别
_, predicted_class = torch.max(probabilities, dim=0)  # 获取概率最高的类别索引
print(f"Predicted class: {class_labels[predicted_class]}")

# 获取 Top-5 类别（如果需要）
top5_prob, top5_catid = torch.topk(probabilities, 5)  # 获取 Top-5 结果
print("Top-5 类别预测结果:")
for i in range(top5_prob.size(0)):
    print(f"{class_labels[top5_catid[i]]}: {top5_prob[i].item():.4f}")

In [1]:
train_accuracies

NameError: name 'train_accuracies' is not defined